In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Creating training set with data from years 2000 to 2017
train_set = pd.concat(
    map(pd.read_csv, ['/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2000.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2001.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2002.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2003.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2004.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2005.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2006.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2007.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2008.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2009.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2010.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2011.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2012.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2013.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2014.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2015.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2016.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2017.csv'
                      ]), ignore_index=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6633998 entries, 0 to 6633997
Data columns (total 29 columns):
 #   Column               Dtype  
---  ------               -----  
 0   State Code           object 
 1   County Code          int64  
 2   Site Num             int64  
 3   Parameter Code       int64  
 4   POC                  int64  
 5   Latitude             float64
 6   Longitude            float64
 7   Datum                object 
 8   Parameter Name       object 
 9   Sample Duration      object 
 10  Pollutant Standard   object 
 11  Date Local           object 
 12  Units of Measure     object 
 13  Event Type           object 
 14  Observation Count    int64  
 15  Observation Percent  float64
 16  Arithmetic Mean      float64
 17  1st Max Value        float64
 18  1st Max Hour         int64  
 19  AQI                  int64  
 20  Method Code          float64
 21  Method Name          object 
 22  Local Site Name      object 
 23  Address              object 
 24

In [5]:
def preprocess(dataFile):
  #Extracting month, day, and year from date
  dataFile['Date Local'] = pd.to_datetime(dataFile['Date Local'])
  dataFile['month'] = dataFile['Date Local'].dt.month
  dataFile['day'] = dataFile['Date Local'].dt.day
  dataFile['year'] = dataFile['Date Local'].dt.year

  dfSpcMethod = pd.DataFrame()
  #Ensuring sample duration is consistent
  dfSpcMethod = dataFile.loc[dataFile['Sample Duration'] == "8-HR RUN AVG BEGIN HOUR"]
  #Dropping extraneous columns
  dfSpcMethodNoNa = dfSpcMethod.drop(columns=['Method Code', 'Date Local', 'Observation Count', 'Observation Percent'])
  #Ensuring State Code has numeric values
  dfSpcMethodNoNa['State Code'] = pd.to_numeric(dfSpcMethodNoNa['State Code'], errors='coerce')
  #Dropping NaN values and resetting index
  dfSpcMethodNoNa = dfSpcMethodNoNa.dropna(subset=['State Code'])
  dfSpcMethodNoNa = dfSpcMethodNoNa.reset_index(drop=True) 
  #Only keeping non-object data types
  dfSpcMethodNoNa = dfSpcMethodNoNa.select_dtypes(exclude=['object'])
  
  return dfSpcMethodNoNa

In [6]:
dfSpcMethodNoNa = pd.DataFrame()
dfSpcMethodNoNa = preprocess(train_set)

In [7]:
#Setting AQI values as our target
y_train = dfSpcMethodNoNa['AQI']
x_train = dfSpcMethodNoNa.drop(columns=['AQI'])

In [8]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6632826 entries, 0 to 6632825
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   State Code       float64
 1   County Code      int64  
 2   Site Num         int64  
 3   Parameter Code   int64  
 4   POC              int64  
 5   Latitude         float64
 6   Longitude        float64
 7   Arithmetic Mean  float64
 8   1st Max Value    float64
 9   1st Max Hour     int64  
 10  month            int64  
 11  day              int64  
 12  year             int64  
dtypes: float64(5), int64(8)
memory usage: 657.9 MB


In [9]:
x_train.head()

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Arithmetic Mean,1st Max Value,1st Max Hour,month,day,year
0,1.0,3,10,44201,1,30.497478,-87.880258,0.023000,0.023,23,2,29,2000
1,1.0,3,10,44201,1,30.497478,-87.880258,0.036471,0.046,11,3,1,2000
2,1.0,3,10,44201,1,30.497478,-87.880258,0.037765,0.062,11,3,2,2000
3,1.0,3,10,44201,1,30.497478,-87.880258,0.034941,0.037,10,3,3,2000
4,1.0,3,10,44201,1,30.497478,-87.880258,0.028882,0.038,11,3,4,2000


In [10]:
x_train['State Code'].isnull().sum()

0

In [11]:
y_train.isnull().sum()

0

In [12]:
#Creating testing set with data from years 2018 to 2020
test_set = pd.concat(
    map(pd.read_csv, ['/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2018.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2019.csv',
                      '/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/daily_44201_2020.csv'
                      ]), ignore_index=True)

In [13]:
nameOfParameter = train_set['Parameter Name'][0]
nameOfParameter

'Ozone'

In [14]:
df2018NoNa = pd.DataFrame()
df2018NoNa = preprocess(test_set)

In [15]:
y_test = df2018NoNa['AQI']
x_test = df2018NoNa.drop(columns = ['AQI'])

In [16]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189467 entries, 0 to 1189466
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   State Code       1189467 non-null  int64  
 1   County Code      1189467 non-null  int64  
 2   Site Num         1189467 non-null  int64  
 3   Parameter Code   1189467 non-null  int64  
 4   POC              1189467 non-null  int64  
 5   Latitude         1189467 non-null  float64
 6   Longitude        1189467 non-null  float64
 7   Arithmetic Mean  1189467 non-null  float64
 8   1st Max Value    1189467 non-null  float64
 9   1st Max Hour     1189467 non-null  int64  
 10  month            1189467 non-null  int64  
 11  day              1189467 non-null  int64  
 12  year             1189467 non-null  int64  
dtypes: float64(4), int64(9)
memory usage: 118.0 MB


In [17]:
#Checking Random Forest Regressor accuracy
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=2)
rfr = rfr.fit(x_train, y_train)
predictedValsRF = rfr.predict(x_test)
print(rfr.score(x_train, y_train))
print(rfr.score(x_test,y_test))

0.8756115944829609
0.8282471811593924


In [18]:
#Checking Linear Regression accuracy
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train, y_train)
predictedValsLinear = lr.predict(x_test)
print(lr.score(x_train, y_train))
print(lr.score(x_test, y_test))

0.8667751814975051
0.8474767432110633


In [19]:
#Checking Bayesian Ridge accuracy
from sklearn.linear_model import BayesianRidge
br = BayesianRidge()
br.fit(x_train, y_train)
predictedValsBR = br.predict(x_test)
print(br.score(x_train, y_train))
print(br.score(x_test, y_test))

0.8667751814971233
0.8474768076685746


In [20]:
#Checking Gradient Boost accuracy
gbr = GradientBoostingRegressor(n_estimators=30)
gbr.fit(x_train, y_train)

GradientBoostingRegressor(n_estimators=30)

In [21]:
values = gbr.predict(x_test)

In [22]:
print(gbr.score(x_train, y_train))
print(gbr.score(x_test, y_test))

0.9937524774317181
0.9928378665232392


In [23]:
#Creating data frame for outputting predicted AQI values
graphDataFrame = x_test[['State Code','County Code','Site Num','Latitude','Longitude','month','day','year']]

In [24]:
#Populating data frame for outputting predicted AQI values with Gradient Boost results
graphDataFrame['AQI predicted'] = values
graphDataFrame['Parameter'] = "Ozone"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [25]:
graphDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189467 entries, 0 to 1189466
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   State Code     1189467 non-null  int64  
 1   County Code    1189467 non-null  int64  
 2   Site Num       1189467 non-null  int64  
 3   Latitude       1189467 non-null  float64
 4   Longitude      1189467 non-null  float64
 5   month          1189467 non-null  int64  
 6   day            1189467 non-null  int64  
 7   year           1189467 non-null  int64  
 8   AQI predicted  1189467 non-null  float64
 9   Parameter      1189467 non-null  object 
dtypes: float64(3), int64(6), object(1)
memory usage: 90.7+ MB


In [26]:
#Outputting data frame containing predicted AQI values to csv
graphDataFrame.to_csv('/content/drive/MyDrive/FA21 CMPE 255 Term Project/Ozone/Ozone_AQI_Predicted', index=False)